In [1]:
from pyspark.sql.types import StructField, StructType, IntegerType, StringType
from pyspark.sql import functions as f

StatementMeta(tcccibele, 6, 1, Finished, Available)

In [2]:
# Definir schema do arquivo
schema = StructType([ 
    StructField("None", StringType(), True), 
    StructField("FaixaEtariaVitima", StringType(), True), 
    StructField("Janeiro", StringType(), True), 
    StructField("Fevereiro", StringType(), True), 
    StructField("Março", StringType(), True), 
    StructField("Abril", StringType(), True), 
    StructField("Maio", StringType(), True), 
    StructField("Junho", StringType(), True), 
    StructField("Julho", StringType(), True), 
    StructField("Agosto", StringType(), True), 
    StructField("Setembro", StringType(), True), 
    StructField("Outubro", StringType(), True), 
    StructField("Novembro", StringType(), True), 
    StructField("Dezembro", StringType(), True), 
    StructField("Total", StringType(), True)
])

StatementMeta(tcccibele, 6, 2, Finished, Available)

In [3]:
# Ler arquivo de origem da landing zone
df = spark.read.format("csv") \
            .schema(schema) \
            .options(quote='"') \
            .load('/landing_zone/2019/Balanco-2019-Ligue180-Idade Vitima.csv')

StatementMeta(tcccibele, 6, 3, Finished, Available)

In [4]:
# Remover colunas e linhas sem valor para o data lake
df = df.where((df[1].isNotNull()) \
                & (df[1] != 'Total') \
                & (df[-1] != 'Total 2019') \
                & (df[-1].isNotNull()))
df = df.drop('None')
df = df.drop('Total')

StatementMeta(tcccibele, 6, 4, Finished, Available)

In [5]:
# Remover "," de todos os campos e substitui null por 0
names = df.schema.names
for name in names:
    if name == 'FaixaEtariaVitima':
        continue
    df = df.withColumn(name,f.regexp_replace(name,',',''))
    df = df.withColumn(name, df[name].cast(IntegerType()))

df = df.fillna(0)

StatementMeta(tcccibele, 6, 5, Finished, Available)

In [6]:
# Escrever dados limpos em formato parque no azure storage
df.write.format('parquet').mode('overwrite').save('/history_zone/2019/IdadeVitima/')

StatementMeta(tcccibele, 6, 6, Finished, Available)

In [7]:
df = spark.read.parquet('/history_zone/2019/IdadeVitima/')
df.toPandas()

StatementMeta(tcccibele, 6, 7, Finished, Available)

,FaixaEtariaVitima,Janeiro,Fevereiro,Março,Abril,Maio,Junho,Julho,Agosto,Setembro,Outubro,Novembro,Dezembro
0,0 a 3 anos,0,0,0,1,0,0,0,1,0,0,0,0
1,4 a 7 anos,1,0,1,0,0,0,0,0,0,0,2,0
2,8 a 11 anos,1,0,0,0,0,0,0,1,2,0,0,0
3,12 a 14 anos,5,2,1,1,1,0,0,1,2,1,1,0
4,15 a 17 anos,13,8,3,9,7,6,0,5,7,7,0,4
5,18 a 24 anos,1556,1296,1440,1275,1303,1062,1107,1282,1241,1249,1196,586
6,25 a 30 anos,1437,1137,1441,1300,1262,1096,1162,1255,1146,1253,1089,576
7,31 a 35 anos,1370,998,1049,946,930,837,843,1017,977,888,881,449
8,36 a 40 anos,1389,1131,1179,1034,1022,896,853,1046,1103,1067,998,488
9,41 a 45 anos,773,774,844,701,734,646,692,819,827,790,655,362
